In [3]:
# ! pip install kaggle
# ! mldir~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download -d shubhambathwal/flight-price-prediction


'mldir~' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.



  0%|          | 0.00/4.16M [00:00<?, ?B/s]
 48%|████▊     | 2.00M/4.16M [00:00<00:00, 11.8MB/s]
 96%|█████████▌| 4.00M/4.16M [00:00<00:00, 11.8MB/s]
100%|██████████| 4.16M/4.16M [00:00<00:00, 11.8MB/s]


In [5]:
# from zipfile import ZipFile
# filename = 'flight-price-prediction.zip'
# with ZipFile(filename, 'r') as zip:
#     zip.extractall()
#     print('Done!')

Done!


In [6]:
import opendatasets as od
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
# %matplotlib inline

from sklearn.model_selection import train_test_split

In [20]:
od.download('https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction/data')

100%|██████████| 4.16M/4.16M [00:00<00:00, 11.8MB/s]


In [7]:
fligheprice = pd.read_csv('Clean_Dataset.csv')